# Prep data for use in project
Kevin Fosnacht 10/19/22

NLP Final

## Import Dependencies

In [1]:
import os, zipfile
import pandas as pd
import glob

## Extract all csvs from zip files

In [2]:
dir_name = 'C:\\Users\\kookie\\OneDrive - Indiana University\\classes\\NLP\\Final\\Democrat'
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        #os.remove(file_name) # delete zipped file
        
dir_name = 'C:\\Users\\kookie\\OneDrive - Indiana University\\classes\\NLP\\Final\\Republican'
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        #os.remove(file_name) # delete zipped file

## Get list of files & append

In [3]:
#join all csvs in the dem folder
path=r'C:\\Users\\kookie\\OneDrive - Indiana University\\classes\\NLP\\Final\\Democrat'
all_files = glob.glob(os.path.join(path, "*.csv"))

dem = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
dem['party']="D"
dem

#join all csvs in the rep folder
path=r'C:\\Users\\kookie\\OneDrive - Indiana University\\classes\\NLP\\Final\\Republican'
all_files = glob.glob(os.path.join(path, "*.csv"))

rep = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
rep['party']="R"
rep

#combine

df=dem.append(rep, ignore_index=True)
df.party.value_counts()

D    93044
R    18994
Name: party, dtype: int64

## Limit data to 2022, combine text columns, remove nulls

In [4]:
df=df[(df['ad_creation_time']>='2022-01-01') | (df['ad_delivery_stop_time']>='2022-01-01')]
df[['ad_creative_bodies','ad_creative_link_titles','ad_creative_link_descriptions']] = df[['ad_creative_bodies','ad_creative_link_titles','ad_creative_link_descriptions']].fillna('')
df['text']=df['ad_creative_bodies']+' '+df['ad_creative_link_titles']+' '+df['ad_creative_link_descriptions']
df['text']=df['text'].str.strip()
df=df[df['text']!='']
df

,ad_archive_id,page_id,page_name,ad_creation_time,ad_delivery_start_time,ad_delivery_stop_time,byline,ad_creative_bodies,ad_creative_link_titles,ad_creative_link_captions,...,impressions,spend,currency,demographic_distribution,delivery_by_region,publisher_platforms,estimated_audience_size,languages,party,text
0,473922708011252,131182018871,Patty Murray,2022-10-06,2022-10-06,NaN,PEOPLE FOR PATTY MURRAY,https://www.usatoday.com/story/news/politics/2...,GOP Senate candidate Tiffany Smiley declines t...,usatoday.com,...,"lower_bound: 0, upper_bound: 999","lower_bound: 0, upper_bound: 99",USD,NaN,NaN,instagram,"lower_bound: 100001, upper_bound: 500000",en,D,https://www.usatoday.com/story/news/politics/2...
1,503011384703565,131182018871,Patty Murray,2022-10-06,2022-10-06,NaN,PEOPLE FOR PATTY MURRAY,I will ALWAYS defend a woman’s right to make h...,Politicians like Tiffany Smiley say abortion i...,thenewstribune.com,...,"lower_bound: 0, upper_bound: 999","lower_bound: 0, upper_bound: 99",USD,"{""age"":""35-44"",""gender"":""female"",""percentage"":...","{""region"":""Washington"",""percentage"":1}",instagram,"lower_bound: 100001, upper_bound: 500000",en,D,I will ALWAYS defend a woman’s right to make h...
2,800775017927348,131182018871,Patty Murray,2022-10-06,2022-10-06,NaN,PEOPLE FOR PATTY MURRAY,Efforts to undermine our fair and free electio...,GOP Washington Senate candidate scrubs website...,axios.com,...,"lower_bound: 4000, upper_bound: 4999","lower_bound: 0, upper_bound: 99",USD,"{""age"":""18-24"",""gender"":""unknown"",""percentage""...","{""region"":""Washington"",""percentage"":1}",instagram,"lower_bound: 100001, upper_bound: 500000",en,D,Efforts to undermine our fair and free electio...
3,5726923204025242,131182018871,Patty Murray,2022-10-04,2022-10-04,NaN,PEOPLE FOR PATTY MURRAY,https://www.usatoday.com/story/news/politics/2...,GOP Senate candidate Tiffany Smiley declines t...,usatoday.com,...,"lower_bound: 0, upper_bound: 999","lower_bound: 0, upper_bound: 99",USD,"{""age"":""18-24"",""gender"":""unknown"",""percentage""...","{""region"":""Alaska"",""percentage"":0.008197},{""re...",instagram,"lower_bound: 50001, upper_bound: 100000",en,D,https://www.usatoday.com/story/news/politics/2...
4,1523241238109743,131182018871,Patty Murray,2022-10-04,2022-10-04,2022-10-04,PEOPLE FOR PATTY MURRAY,I will ALWAYS defend a woman’s right to make h...,Politicians like Tiffany Smiley say abortion i...,thenewstribune.com,...,"lower_bound: 0, upper_bound: 999","lower_bound: 0, upper_bound: 99",USD,NaN,NaN,instagram,"lower_bound: 100001, upper_bound: 500000",en,D,I will ALWAYS defend a woman’s right to make h...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111909,899373557378669,35541499994,Dr. Mehmet Oz,2022-01-12,2022-01-12,2022-01-31,DOCTOR OZ FOR SENATE,It is time to put the people in control of the...,Make Medicine Affordable,doctoroz.com,...,"lower_bound: 500000, upper_bound: 599999","lower_bound: 4000, upper_bound: 4499",USD,"{""age"":""13-17"",""gender"":""female"",""percentage"":...","{""region"":""Pennsylvania"",""percentage"":1}","facebook,instagram",lower_bound: 1000001,en,R,It is time to put the people in control of the...
111910,4713611172025637,35541499994,Dr. Mehmet Oz,2022-01-12,2022-01-12,2022-01-31,DOCTOR OZ FOR SENATE,"As a Doctor and TV host, I took on the medical...",Time to Fight Back,doctoroz.com,...,"lower_bound: 700000, upper_bound: 799999","lower_bound: 6000, upper_bound: 6999",USD,"{""age"":""13-17"",""gender"":""female"",""percentage"":...","{""region"":""Pennsylvania"",""percentage"":1}","facebook,instagram",lower_bound: 1000001,en,R,"As a Doctor and TV host, I took on the medical..."
111911,1106384986841367,35541499994,Dr. Mehmet Oz,2022-01-12,2022-01-12,2022-01-31,DOCTOR OZ FOR SENATE,It is time we set Washington straight on their...,A COVID Nightmare,doctoroz.com,...,"lower_bound: 100000, upper_bound: 124999","lower_bound: 800, upper_bound: 899",USD,"{""age"":""18-

## apply normalization function

In [5]:
import nltk
import text_normalizer as tn

stopword_list = nltk.corpus.stopwords.words('english')
# just to keep negation if any in bi-grams
stopword_list.append('senate')

# normalize our corpus
norm_corpus = tn.normalize_corpus(corpus=df['text'], html_stripping=True, contraction_expansion=True, 
                                  accented_char_removal=True, text_lower_case=True, text_lemmatization=True, 
                                  text_stemming=False, special_char_removal=True, remove_digits=True,
                                  stopword_removal=True, stopwords=stopword_list)
df['normtext'] = norm_corpus
df


C:\Users\kookie\anaconda3\lib\site-packages\bs4\__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


,ad_archive_id,page_id,page_name,ad_creation_time,ad_delivery_start_time,ad_delivery_stop_time,byline,ad_creative_bodies,ad_creative_link_titles,ad_creative_link_captions,...,spend,currency,demographic_distribution,delivery_by_region,publisher_platforms,estimated_audience_size,languages,party,text,normtext
0,473922708011252,131182018871,Patty Murray,2022-10-06,2022-10-06,NaN,PEOPLE FOR PATTY MURRAY,https://www.usatoday.com/story/news/politics/2...,GOP Senate candidate Tiffany Smiley declines t...,usatoday.com,...,"lower_bound: 0, upper_bound: 99",USD,NaN,NaN,instagram,"lower_bound: 100001, upper_bound: 500000",en,D,https://www.usatoday.com/story/news/politics/2...,httpswww usatoday comstorynewspoliticstiffanys...
1,503011384703565,131182018871,Patty Murray,2022-10-06,2022-10-06,NaN,PEOPLE FOR PATTY MURRAY,I will ALWAYS defend a woman’s right to make h...,Politicians like Tiffany Smiley say abortion i...,thenewstribune.com,...,"lower_bound: 0, upper_bound: 99",USD,"{""age"":""35-44"",""gender"":""female"",""percentage"":...","{""region"":""Washington"",""percentage"":1}",instagram,"lower_bound: 100001, upper_bound: 500000",en,D,I will ALWAYS defend a woman’s right to make h...,always defend woman right make health care dec...
2,800775017927348,131182018871,Patty Murray,2022-10-06,2022-10-06,NaN,PEOPLE FOR PATTY MURRAY,Efforts to undermine our fair and free electio...,GOP Washington Senate candidate scrubs website...,axios.com,...,"lower_bound: 0, upper_bound: 99",USD,"{""age"":""18-24"",""gender"":""unknown"",""percentage""...","{""region"":""Washington"",""percentage"":1}",instagram,"lower_bound: 100001, upper_bound: 500000",en,D,Efforts to undermine our fair and free electio...,effort undermine fair free election put democr...
3,5726923204025242,131182018871,Patty Murray,2022-10-04,2022-10-04,NaN,PEOPLE FOR PATTY MURRAY,https://www.usatoday.com/story/news/politics/2...,GOP Senate candidate Tiffany Smiley declines t...,usatoday.com,...,"lower_bound: 0, upper_bound: 99",USD,"{""age"":""18-24"",""gender"":""unknown"",""percentage""...","{""region"":""Alaska"",""percentage"":0.008197},{""re...",instagram,"lower_bound: 50001, upper_bound: 100000",en,D,https://www.usatoday.com/story/news/politics/2...,httpswww usatoday comstorynewspoliticstiffanys...
4,1523241238109743,131182018871,Patty Murray,2022-10-04,2022-10-04,2022-10-04,PEOPLE FOR PATTY MURRAY,I will ALWAYS defend a woman’s right to make h...,Politicians like Tiffany Smiley say abortion i...,thenewstribune.com,...,"lower_bound: 0, upper_bound: 99",USD,NaN,NaN,instagram,"lower_bound: 100001, upper_bound: 500000",en,D,I will ALWAYS defend a woman’s right to make h...,always defend woman right make health care dec...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111909,899373557378669,35541499994,Dr. Mehmet Oz,2022-01-12,2022-01-12,2022-01-31,DOCTOR OZ FOR SENATE,It is time to put the people in control of the...,Make Medicine Affordable,doctoroz.com,...,"lower_bound: 4000, upper_bound: 4499",USD,"{""age"":""13-17"",""gender"":""female"",""percentage"":...","{""region"":""Pennsylvania"",""percentage"":1}","facebook,instagram",lower_bound: 1000001,en,R,It is time to put the people in control of the...,time put people control health insane drug pri...
111910,4713611172025637,35541499994,Dr. Mehmet Oz,2022-01-12,2022-01-12,2022-01-31,DOCTOR OZ FOR SENATE,"As a Doctor and TV host, I took on the medical...",Time to Fight Back,doctoroz.com,...,"lower_bound: 6000, upper_bound: 6999",USD,"{""age"":""13-17"",""gender"":""female"",""percentage"":...","{""region"":""Pennsylvania"",""percentage"":1}","facebook,instagram",lower_bound: 1000001,en,R,"As a Doctor and TV host, I took on the medical...",doctor tv host take medical establishment sena...
111911,1106384986841367,35541499994,Dr. Mehmet Oz,2022-01-12,2022-01-12,2022-01-31,DOCTOR OZ FOR SENATE,It is time we set Washington straight on their...,A COVID Nightmare,doctoroz.com,...,"lower_bound: 800

## Limit to data that is needed

In [12]:
df=df[['party','page_name','text','normtext']]
df=df[df['normtext'].notna()] #dropping any additional NAs post normalization

## Save out

In [7]:
df.to_csv('C:\\Users\\kookie\\OneDrive - Indiana University\\classes\\NLP\\Final\\fb_ads.csv', index=False)

In [13]:
df.shape

(40732, 4)

In [11]:
df[df['normtext'].isna()]

,party,page_name,text,normtext
